In [7]:
import pickle

f = open('course_selections.pkl','rb')
selections = pickle.load(f)

f = open('student_acad_programs.pkl','rb')
students = pickle.load(f)

f = open('module_attrs.pkl', 'rb')
modules = pickle.load(f)

In [12]:
list(modules['CS1010'].values())[-1]

{'ModuleCode': 'CS1010',
 'ModuleTitle': 'Programming Methodology',
 'AcadYear': '2018/2019',
 'SemesterName': 'Special Term II',
 'Faculty': 'SCHOOL OF COMPUTING',
 'Department': 'COMPUTER SCIENCE',
 'ModuleDescription': 'This module introduces the fundamental concepts of problem solving by computing and programming using an imperative programming language. It is the first and foremost introductory course to computing.  Topics covered include computational thinking and computational problem solving, designing and specifying an algorithm, basic problem formulation and problem solving approaches, program development, coding, testing and debugging, fundamental programming constructs (variables, types, expressions, assignments, functions, control structures, etc.), fundamental data structures (arrays, strings, composite data types), basic sorting, and recursion.',
 'CrossModule': '',
 'ModuleCredit': '4',
 'Workload': '2-1-1-3-3',
 'Prerequisite': 'Nil',
 'Preclusion': 'CS1010E, CS1010J, 

In [32]:
import requests
from collections import defaultdict

modules = defaultdict(defaultdict)

for i in range(13,20):
    for j in range(5):
        if i == 19 and j != 1:
            continue
        URL = f"https://api.nusmods.com/20{i}-20{i+1}/{j}/bulletinModulesRaw.json"    
        response = requests.get(URL)
        if response.status_code == 200:
            json = response.json()
            for mod in json:
                modules[mod['ModuleCode']][mod['AcadYear']] = mod
        else:
            print(URL, "not found.")

In [39]:
# import pickle
# f = open("module_attrs.pkl", 'wb')
# pickle.dump(modules, f)

In [3]:
# Construct prompts: zero-shot
# 'ModuleCode': 'CS1010',
            #   'ModuleTitle': 'Programming Methodology',
            #   'AcadYear': '2018/2019',
            #   'SemesterName': 'Special Term II',
            #   'Faculty': 'SCHOOL OF COMPUTING',
            #   'Department': 'COMPUTER SCIENCE',
            #   'ModuleDescription': 'This module introduces the fundamental concepts of problem solving by computing and programming using an imperative programming language. It is the first and foremost introductory course to computing.  Topics covered include computational thinking and computational problem solving, designing and specifying an algorithm, basic problem formulation and problem solving approaches, program development, coding, testing and debugging, fundamental programming constructs (variables, types, expressions, assignments, functions, control structures, etc.), fundamental data structures (arrays, strings, composite data types), basic sorting, and recursion.',
            #   'CrossModule': '',
            #   'ModuleCredit': '4',
            #   'Workload': '2-1-1-3-3',
            #   'Prerequisite': 'Nil',
            #   'Preclusion': 'CS1010E, CS1010J, CS1010S, CS1010X, CS1010XCP, CS1101S',
            #   'Corequisite': 'Nil'

from collections import defaultdict
def gen_course_table(mods):
    mods = mods[-15:]
    import random
    random.shuffle(mods)
    columns = """
    |Course Code|Course Title|Academic Year|Faculty|Department|ModuleCredit|Workload|Prerequisite|Preclusion|Corequisite|"""
    
    for (term, mod) in mods:
        if mod not in modules:
            continue
        if term not in modules[mod]:
            mod_info = list(modules[mod].values())[-1]
        else:
            mod_info = modules[mod][term]
        mi = defaultdict(str, mod_info)
        columns += f"""
        |{mi['ModuleCode']}|{mi['ModuleTitle']}|{mi['AcadYear']}|{mi['Faculty']}|{mi['Department']}|{mi['ModuleCredit']}|{mi['Workload']}|{mi['Prerequisite']}|{mi['Preclusion']}|{mi['Corequisite']}|"""
        
    return columns

def generate_prompt(courses, selections):
    if input:
        return f"""Below is a course recommendation task. Fill up the Selection Table (<TO FILL>) based on the likelihood of the student's next selections.

### Course Table:
{courses}

### Selection Table:
{selections}

### Filled Selection Table:"""

prompts = []

for (stu, mods) in selections.items():
    mods = sorted(list(mods), key=lambda x:x[0])
    prev = mods[:int(len(mods) * 0.9)]
    pred = mods[int(len(mods) * 0.9):]
    if not prev or not pred:
        continue
    if stu not in students:
        continue
    if prev[-1][0] in students[stu]:
        desc = students[stu][prev[-1][0]]
    else:
        desc = sorted(list(students[stu]))[-1][1]
    selection_table = f"""
        |Major Description|Past Course Selections in Time Order|Predicted Course Selections|
        |{desc}|{list(map(lambda x:x[1], prev))}| <TO FILL>|

    """  
#        Correct answers to fill up the table: {list(map(lambda x:x[1], pred))}   
    course_table = gen_course_table(mods)
    prompt = generate_prompt(course_table, selection_table)
    prompts.append(prompt)

In [86]:
print(max(map(len, prompts)))

9617


In [7]:
print(prompts[11])

Below is a course recommendation task. Fill up the Selection Table (<TO FILL>) based on the likelihood of the student's next selections.

### Course Table:

    |Course Code|Course Title|Academic Year|Faculty|Department|ModuleCredit|Workload|Prerequisite|Preclusion|Corequisite|
        |PS5313R|SEMINAR ON STATE AND SOCIETY|2014/2015|ARTS & SOCIAL SCIENCES|POLITICAL SCIENCE|5|0-3-0-2-5||||
        |SN3281|The Story of Indian Business|2018/2019|ARTS & SOCIAL SCIENCES|SOUTH ASIAN STUDIES PROGRAMME|4|2-1-0-3-4||||
        |SW1101E|Social Work: A Heart-Head-Hand Connection|2018/2019|ARTS & SOCIAL SCIENCES|SOCIAL WORK|4|2-1-0-1-6|Nil|NIL|NIL|
        |SN3274|South Asian Cinema|2018/2019|ARTS & SOCIAL SCIENCES|SOUTH ASIAN STUDIES PROGRAMME|4|2-1-0-2-5||||
        |PS2255|Politics of the Middle East|2018/2019|ARTS & SOCIAL SCIENCES|POLITICAL SCIENCE|4|2-1-0-2-5|NIL|GEK2025|NIL|
        |PS4225|Contemporary Issues in Chinese Politics|2015/2016|ARTS & SOCIAL SCIENCES|POLITICAL SCIENCE|5|0-3-0-2-

In [4]:
import json

json.dump({i+1: prompt for (i, prompt) in enumerate(prompts[:100])}, open("exp_zshot.json", "w"), indent = 4)